<a href="https://colab.research.google.com/github/Nithesh-b/Twitter_Sentiment/blob/post-viva/CustomerAnalytics_LSTMusingKeras_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



###  Binary Classification
## Keras for vectorization of clean tweet and apply LSTM

#### 1. we will focus on following 3 columns
   ###### a) Tweet id  ,  
   ###### b) Clean-tweet
   ###### c) class ( derived from Text blob)
   

### Script flow 
##### Step-1
###### Read the data from folder  ( Tweet id , Clean-tweet and  class ( derived from Text blob)
###### Split the data Train and Test 
###### Write Train and Test data in to separete csv files 

##### Step-2
###### Read train and test csv file
###### Clean tweet text of  both test and train file using keras.tokenizer
###### Build the LSTM model on train and apply it on test 




In [1]:
import numpy as np 
import pandas as pd 
dataFolder = "/content/sample_data/outputs"


In [2]:
tweet_df = pd.read_csv(dataFolder+"/Final_PreProcessing_Group33_Cleaned_Tweets.csv")

In [3]:
tweet_df.head(5)

,tweet_id,Tweet,class
0,neu-GG-Tweet-11942,ahh repli random follow do not how sad haha,1
1,neu-GG-Tweet-11941,awwww did not get hero,0
2,neu-GG-Tweet-11940,oh realli what bummer,1
3,neu-GG-Tweet-11937,wtf whyd flag oop shitti mous click star,1
4,neu-GG-Tweet-11930,stick intel drive one given decent review anan...,1


In [4]:
print(tweet_df['class'].value_counts())

1    2801
0    2623
Name: class, dtype: int64


##Train test split ( Tweet-id , Clean_tweet , class)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test , tweet_id_train , tweet_id_test = train_test_split(tweet_df['Tweet'], tweet_df['class'], tweet_df['tweet_id'],test_size = 0.20, random_state = 100 , stratify = tweet_df['class'])    


## Store Train and Test data to csv file - so that we can use those in different ML model

In [6]:
# Create Train Data frame
train_dataset = pd.DataFrame({
    'tweet_id' : tweet_id_train ,
    'Tweet'    : X_train,
    'class'    : y_train
    })

print(train_dataset['class'].value_counts())



1    2241
0    2098
Name: class, dtype: int64


In [7]:
# Create Train Test frame
test_dataset = pd.DataFrame({
    'tweet_id' : tweet_id_test ,
    'Tweet'    : X_test,
    'class': y_test
    })

print(test_dataset['class'].value_counts())

1    560
0    525
Name: class, dtype: int64


In [8]:
# Save Train Data
train_dataset.to_csv(dataFolder+"/LSTM_train_Data.csv", index = False)

# Save test data
test_dataset.to_csv(dataFolder+"/LSTM_test_Data.csv", index = False)

### Keras Library 

In [9]:
# Import Tensorflow
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

## Keras Tokenizer
###### There are many different methods to do this conversion like count vectorizer, TF-IDF vectorizer. We will use Keras tokenizers to serve the same purpose.

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd

In [11]:
dataFolder = "/content/sample_data/outputs"
# Load data

train_data = pd.read_csv(dataFolder+"/LSTM_train_Data.csv")

test_data = pd.read_csv(dataFolder+"/LSTM_test_Data.csv")

In [12]:
train_data.columns

Index(['tweet_id', 'Tweet', 'class'], dtype='object')

##Tokenization

In [13]:
tokenizer = Tokenizer(num_words = 2500, split = ' ')

# Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(train_data['Tweet'].astype(str).values)

train_tweets = tokenizer.texts_to_sequences(train_data['Tweet'].astype(str).values)


max_len = max([len(i) for i in train_tweets])

train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(test_data['Tweet'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)

In [14]:
train_tweets

array([[   0,    0,    0, ...,  354,  289,  859],
       [   0,    0,    0, ...,    3,   34,  686],
       [   0,    0,    0, ...,    3,    8, 1368],
       ...,
       [   0,    0,    0, ...,  124,   31,   99],
       [   0,    0,    0, ...,   32,   10,  154],
       [   0,    0,    0, ...,   36,  226, 1754]], dtype=int32)

In [15]:
# Building the model
model = Sequential()
model.add(Embedding(2500, 128, input_length = train_tweets.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout = 0.2))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 23, 128)           320000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 23, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 714,754
Trainable params: 714,754
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Training the model
history = model.fit(train_tweets, pd.get_dummies(train_data['class']).values, epochs = 10, batch_size = 128, validation_split = 0.2)

Epoch 1/10
28/28 [==============================] - 6s 229ms/step - loss: 0.6742 - accuracy: 0.6266 - val_loss: 0.6013 - val_accuracy: 0.6382
Epoch 2/10
28/28 [==============================] - 6s 217ms/step - loss: 0.4225 - accuracy: 0.8401 - val_loss: 0.3525 - val_accuracy: 0.8583
Epoch 3/10
28/28 [==============================] - 6s 222ms/step - loss: 0.2194 - accuracy: 0.9167 - val_loss: 0.2476 - val_accuracy: 0.9078
Epoch 4/10
28/28 [==============================] - 6s 222ms/step - loss: 0.1204 - accuracy: 0.9597 - val_loss: 0.2099 - val_accuracy: 0.9217
Epoch 5/10
28/28 [==============================] - 6s 222ms/step - loss: 0.0806 - accuracy: 0.9726 - val_loss: 0.2491 - val_accuracy: 0.9194
Epoch 6/10
28/28 [==============================] - 6s 224ms/step - loss: 0.0564 - accuracy: 0.9816 - val_loss: 0.2353 - val_accuracy: 0.9240
Epoch 7/10
28/28 [==============================] - 6s 225ms/step - loss: 0.0468 - accuracy: 0.9862 - val_loss: 0.3196 - val_accuracy: 0.9044
Epoch 

In [17]:
# Testing the model

score, accuracy = model.evaluate(test_tweets, pd.get_dummies(test_data['class']).values, batch_size = 128)
print("Test accuracy: {}".format(accuracy))

9/9 [==============================] - 0s 49ms/step - loss: 0.4872 - accuracy: 0.8857
Test accuracy: 0.8857142925262451
